<a href="https://colab.research.google.com/github/prakash888kp/Ensemble-XGB-LIGHTGBM-for-crop-yield-prediction/blob/main/Ensemble%20XGBoost%20and%20LightGBM%20crop%20yield%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error



In [46]:
# Load the data from a CSV file
data = pd.read_csv('/content/drive/MyDrive/dataset/kaggle_sub1/test.csv')



In [47]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [48]:
data

,id,clonesize,honeybee,bumbles,andrena,osmia,MaxOfUpperTRange,MinOfUpperTRange,AverageOfUpperTRange,MaxOfLowerTRange,MinOfLowerTRange,AverageOfLowerTRange,RainingDays,AverageRainingDays,fruitset,fruitmass,seeds
0,15289,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.399367,0.408088,31.394569
1,15290,12.5,0.25,0.25,0.75,0.63,94.6,57.2,79.0,68.2,33.0,55.9,1.0,0.10,0.488048,0.442866,36.846956
2,15291,12.5,0.25,0.25,0.63,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.583379,0.487057,40.037644
3,15292,25.0,0.50,0.38,0.38,0.63,86.0,52.0,71.9,62.0,30.0,50.8,16.0,0.26,0.433014,0.422847,33.116091
4,15293,37.5,0.75,0.25,0.25,0.25,94.6,57.2,79.0,68.2,33.0,55.9,24.0,0.39,0.360996,0.388860,29.558019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10189,25478,25.0,0.25,0.25,0.25,0.25,86.0,52.0,71.9,62.0,30.0,50.8,24.0,0.39,0.474162,0.437923,34.525258
10190,25479,25.0,0.50,0.25,0.50,0.75,77.4,46.8,64.7,55.8,27.0,45.8,16.0,0.26,0.482854,0.440676,35.648221
10191,25480,25.0,0.50,0.38,0.50,0.50,77.4,46.8,64.7,55.8,27.0,45.8,16.0,0.26,0.568854,0.463065,37.724724
10192,25481,12.5,0.25,0.25,0.38,0.50,94.6,57.2,79.0,68.2,33.0,55.9,34.0,0.56,0.407374,0.409261,31.881847


In [49]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data.drop('id', axis=1), data['fruitmass'], test_size=0.2, random_state=42)



In [50]:
# Optimize hyperparameters for XGBoost
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'max_depth': [3, 5, 7, 9],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'subsample': [0.5, 0.7, 1.0]
}
xgb_grid = GridSearchCV(xgb_model, xgb_params, scoring='neg_mean_absolute_error')
xgb_grid.fit(train_data, train_labels)
print('XGBoost Best Params:', xgb_grid.best_params_)



XGBoost Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}


In [51]:
# Optimize hyperparameters for LightGBM
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'max_depth': [3, 5, 7, 9],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'subsample': [0.5, 0.7, 1.0]
}
lgb_grid = GridSearchCV(lgb_model, lgb_params, scoring='neg_mean_absolute_error')
lgb_grid.fit(train_data, train_labels)
print('LightGBM Best Params:', lgb_grid.best_params_)



LightGBM Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}


In [52]:
# Create a stacking ensemble with optimized XGBoost and LightGBM as base models
estimators = [
    ('xgb', xgb.XGBRegressor(**xgb_grid.best_params_, random_state=42)),
    ('lgb', lgb.LGBMRegressor(**lgb_grid.best_params_, random_state=42))
]
stacking_model = StackingRegressor(
    estimators=estimators, final_estimator=LinearRegression())



In [53]:
# Train the stacking model on the training data
stacking_model.fit(train_data, train_labels)



StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=1.0,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rat...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=7,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...)),
                              ('lgb',
                               LGBMRegressor(learning_rate=0.05, max_depth=7,
                                             n_estimators=200, random_state=42,
                                             subsample=0.5))],
                  final_estimator=LinearRegression())

In [54]:
# Make predictions on the testing data using the stacking model
ensemble_preds = stacking_model.predict(test_data)

# Calculate the mean absolute error of the ensemble predictions
mae = mean_absolute_error(test_labels, ensemble_preds)

print('Mean Absolute Error:', mae)

Mean Absolute Error: 2.285789119056074e-05
